This notebook allows collecting results across multiple pipeline runs. Concatenating across sessions and bin sizes. 
It takes CSVs, then determines the most recent one from the filename. 

There are two major sets of decoded epochs - Laps and Ripples/Replays
There are two sets of marginals for the decoded epochs - the "by epoch" and "by time bin" marginals.
The "by time bin" epochs are a larger granulation, with each epoch consisting of one or more time bin.


```python

# 2024-01-23 - 
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-fet11-01_12-58-54_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-11-03_12-3-25_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-11-02_17-46-44_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-11-02_19-28-0_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-4-10_12-58-3_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-4-10_12-25-50_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-4-09_16-40-54_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-09_22-24-40_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-12_16-53-46_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-4-09_17-29-30_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-08_14-26-15_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-09_1-22-43_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-07_16-40-19_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-08_21-16-25_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-12_15-55-31_time_bin_size_sweep_results.h5



# found_session_export_paths = [Path(v).resolve() for v in  ["C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0420PM-kdiba_pin01_one_fet11-01_12-58-54-(laps_marginals_df).csv",
# "C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0420PM-kdiba_pin01_one_fet11-01_12-58-54-(ripple_marginals_df).csv",
# "C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0645PM-kdiba_pin01_one_fet11-01_12-58-54-(laps_marginals_df).csv",
# "C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0645PM-kdiba_pin01_one_fet11-01_12-58-54-(ripple_marginals_df).csv",
# "C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0828PM-kdiba_pin01_one_fet11-01_12-58-54-(laps_marginals_df).csv",
# "C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0828PM-kdiba_pin01_one_fet11-01_12-58-54-(ripple_marginals_df).csv",
# ]]

[
    "2024-02-19_0430PM-kdiba_vvp01_one_2006-4-10_12-25-50-(ripple_simple_pf_pearson_merged_df)_tbin-0.025.csv",
    "2024-02-17_0210AM-kdiba_vvp01_one_2006-4-09_17-29-30-(laps_simple_pf_pearson_merged_df)_tbin-0.025.csv",
    "2024-02-19_0430PM-kdiba_vvp01_one_2006-4-10_12-25-50-(laps_weighted_corr_merged_df)_tbin-0.025.csv",
    "2024-02-19_0430PM-kdiba_vvp01_one_2006-4-10_12-25-50-(ripple_weighted_corr_merged_df)_tbin-0.025.csv",
]

```

In [1]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3

# required to enable non-blocking interaction:
%gui qt5

import sys
from copy import deepcopy
from typing import Dict, List, Tuple, Optional, Callable, Union, Any
from typing_extensions import TypeAlias
from nptyping import NDArray
import neuropy.utils.type_aliases as types
from collections import defaultdict

import numpy as np
import pandas as pd
import re
from pathlib import Path
from datetime import datetime

from neuropy.utils.indexing_helpers import PandasHelpers
from pyphocorehelpers.indexing_helpers import partition_df
# Set the maximum number of columns to display
pd.set_option('display.max_columns', 100)

# Plotting
# import pylustrator # customization of figures
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
_bak_rcParams = mpl.rcParams.copy()

matplotlib.use('Qt5Agg')
# %matplotlib inline
# %matplotlib auto

# Switch to the desired interactivity mode
plt.interactive(True)

# _restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
# _restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
template: str = 'plotly_dark' # set plotl template
pio.templates.default = template
from pyphocorehelpers.plotting.media_output_helpers import fig_to_clipboard

from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import _helper_build_figure, plotly_pre_post_delta_scatter, plotly_pre_post_delta_scatter, plot_across_sessions_scatter_results

# from ..PendingNotebookCode import plot_across_sessions_scatter_results, plot_histograms, plot_stacked_histograms
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import find_csv_files, find_HDF5_files, find_most_recent_files, process_csv_file, plot_across_sessions_scatter_results, plot_histograms, plot_stacked_histograms
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DecoderDecodedEpochsResult


debug_print: bool = False

TODAY_DAY_DATE: str = f"2024-03-21_Mac"

print(f'TODAY_DAY_DATE: {TODAY_DAY_DATE}')

types.session_str: TypeAlias = str # a unique session identifier

def process_and_load_file(session_dict, df_file_name_key, loaded_dict, session_name, curr_session_t_delta, time_key):
    try:
        file_path = session_dict[df_file_name_key]
        loaded_dict[session_name] = process_csv_file(file_path, session_name, curr_session_t_delta, time_key)
    except BaseException as e:
        print(f'session {session_name} did not fully work. (error {e}. Skipping.')

def _common_cleanup_operations(a_df):
    """ post loading and concatenation across sessions dataframe cleanup """
    if a_df is None:
        return None
    ## Drop the weird 'Unnamed: 0' column:
    # Rename column 'Unnamed: 0' to 'abs_time_bin_index'
    a_df = a_df.rename(columns={'Unnamed: 0': 'abs_time_bin_index'})
    # Drop column: 'abs_time_bin_index'
    a_df = a_df.drop(columns=['abs_time_bin_index'])
    # Add additional 'epoch_idx' column for compatibility:
    if 'epoch_idx' not in a_df:
        if 'lap_idx' in a_df:
            a_df['epoch_idx'] = a_df['lap_idx']
        if 'ripple_idx' in a_df:
            a_df['epoch_idx'] = a_df['ripple_idx']
    return a_df

Automatic pdb calling has been turned OFF
build_module_logger(module_name="Spike3D.pipeline"):
	 Module logger com.PhoHale.Spike3D.pipeline has file logging enabled and will log to EXTERNAL/TESTING/Logging/debug_com.PhoHale.Spike3D.pipeline.log
TODAY_DAY_DATE: 2024-03-21_Mac


# Definitions
"_new" (as in `all_sessions_new_ripple_df`) adds simple_pearson and a few other measures

In [2]:
## Load across session t_delta CSV, which contains the t_delta for each session:

# cuttoff_date = datetime(2024, 3, 16)
cuttoff_date = None

# t_delta_csv_path = Path(r'C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\output\collected_outputs\2024-01-18_GL_t_split_df.csv').resolve() # Apogee
# t_delta_csv_path = Path('/home/halechr/cloud/turbo/Data/Output/collected_outputs/2024-01-18_GL_t_split_df.csv').resolve() # GL

# collected_outputs_directory = '/home/halechr/FastData/collected_outputs/'
# collected_outputs_directory = r'C:\Users\pho\Desktop\collected_outputs'
# collected_outputs_directory = r'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs' # APOGEE
# collected_outputs_directory = '/home/halechr/cloud/turbo/Data/Output/collected_outputs' # GL

known_collected_outputs_paths = [Path(v).resolve() for v in ['/Users/pho/Dropbox (University of Michigan)/MED-DibaLabDropbox/Data/Pho/Outputs/output/collected_outputs', r'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs', '/home/halechr/FastData/collected_outputs/', '/home/halechr/cloud/turbo/Data/Output/collected_outputs']]
collected_outputs_directory = find_first_extant_path(known_collected_outputs_paths)
assert collected_outputs_directory.exists(), f"collected_outputs_directory: {collected_outputs_directory} does not exist! Is the right computer's config commented out above?"
# fullwidth_path_widget(scripts_output_path, file_name_label='Scripts Output Path:')
print(f'collected_outputs_directory: {collected_outputs_directory}')

t_delta_csv_path = collected_outputs_directory.joinpath('2024-01-18_GL_t_split_df.csv').resolve() # GL
assert t_delta_csv_path.exists()

## Find the files:
csv_files = find_csv_files(collected_outputs_directory)
h5_files = find_HDF5_files(collected_outputs_directory)

csv_sessions = find_most_recent_files(found_session_export_paths=csv_files, cuttoff_date=cuttoff_date)
h5_sessions = find_most_recent_files(found_session_export_paths=h5_files)

## The CSV containing the session delta time:
t_delta_df = pd.read_csv(t_delta_csv_path, index_col=0) # Assuming that your CSV file has an index column
# adds `delta_aligned_t_start`, `delta_aligned_t_end` columns
t_delta_df['delta_aligned_t_start'] = t_delta_df['t_start'] - t_delta_df['t_delta']
t_delta_df['delta_aligned_t_end'] = t_delta_df['t_end'] - t_delta_df['t_delta']

# computes `earliest_delta_aligned_t_start`, latest_delta_aligned_t_end
earliest_delta_aligned_t_start: float = np.nanmin(t_delta_df['delta_aligned_t_start'])
latest_delta_aligned_t_end: float = np.nanmax(t_delta_df['delta_aligned_t_end'])
print(f'earliest_delta_aligned_t_start: {earliest_delta_aligned_t_start}, latest_delta_aligned_t_end: {latest_delta_aligned_t_end}')
t_delta_dict = t_delta_df.to_dict(orient='index')
# t_delta_df


# #TODO 2024-03-02 12:12: - [ ] Could add weighted correlation if there is a dataframe for that and it's computed:
_df_raw_variable_names = ['simple_pf_pearson_merged_df', 'weighted_corr_merged_df']
_df_variables_names = ['laps_weighted_corr_merged_df', 'ripple_weighted_corr_merged_df', 'laps_simple_pf_pearson_merged_df', 'ripple_simple_pf_pearson_merged_df']

# # tbin_values_dict = {'laps': self.laps_decoding_time_bin_size, 'ripple': self.ripple_decoding_time_bin_size}
time_col_name_dict = {'laps': 'lap_start_t', 'ripple': 'ripple_start_t'} ## default should be 't_bin_center'

# fold older files:
# {'laps_marginals_df': 'lap_start_t', 'ripple_marginals_df': 'ripple_start_t', 'laps_time_bin_marginals_df':'t_bin_center', 'ripple_time_bin_marginals_df':'t_bin_center'}
    

# csv_sessions
# Extract each of the separate files from the sessions:

# final_sessions: Dict[types.session_str, Dict[str, Path]] = {}

final_sessions_loaded_laps_dict = {}
final_sessions_loaded_ripple_dict = {}
final_sessions_loaded_laps_time_bin_dict = {}
final_sessions_loaded_ripple_time_bin_dict = {}

final_sessions_loaded_simple_pearson_laps_dict = {}
final_sessions_loaded_simple_pearson_ripple_dict = {}

# stupid reudndant but compatible method
final_sessions_loaded_laps_wcorr_dict = {}
final_sessions_loaded_ripple_wcorr_dict = {}

final_sessions_loaded_laps_all_scores_dict = {}
final_sessions_loaded_ripple_all_scores_dict = {}


# final_sessions_loaded_nested_dict = {} # to be filled like
# {
#     'laps': {},
#     'ripple': {},
# }


if cuttoff_date is not None:
    final_sessions: Dict[types.session_str, Dict[str, Path]] = {session_str:{file_type:a_path for file_type, (an_decoding_time_bin_size_str, a_path, an_export_datetime) in session_dict.items() if an_export_datetime >= cuttoff_date}
                                                                                              for session_str, session_dict in csv_sessions.items() }
else:
    # no cutoff recency date:
    final_sessions: Dict[types.session_str, Dict[str, Path]] = {session_str:{file_type:a_path for file_type, (an_decoding_time_bin_size_str, a_path, an_export_datetime) in session_dict.items()}
                                                                                            for session_str, session_dict in csv_sessions.items()}


for session_str, session_dict in final_sessions.items():
    session_name = str(session_str)  # Extract session name from the filename
    if debug_print:
        print(f'processing session_name: {session_name}')
    curr_session_t_delta = t_delta_dict.get(session_name, {}).get('t_delta', None)
    if curr_session_t_delta is None:
        print(f'WARN: curr_session_t_delta is None for session_str = "{session_str}"')

    # Process each file type with its corresponding details
    process_and_load_file(session_dict, 'laps_marginals_df', final_sessions_loaded_laps_dict, session_str, curr_session_t_delta, 'lap_start_t')
    process_and_load_file(session_dict, 'ripple_marginals_df', final_sessions_loaded_ripple_dict, session_str, curr_session_t_delta, 'ripple_start_t')
    process_and_load_file(session_dict, 'laps_time_bin_marginals_df', final_sessions_loaded_laps_time_bin_dict, session_str, curr_session_t_delta, 't_bin_center')
    process_and_load_file(session_dict, 'ripple_time_bin_marginals_df', final_sessions_loaded_ripple_time_bin_dict, session_str, curr_session_t_delta, 't_bin_center')
    process_and_load_file(session_dict, 'laps_simple_pf_pearson_merged_df', final_sessions_loaded_simple_pearson_laps_dict, session_str, curr_session_t_delta, 'lap_start_t')
    process_and_load_file(session_dict, 'ripple_simple_pf_pearson_merged_df', final_sessions_loaded_simple_pearson_ripple_dict, session_str, curr_session_t_delta, 'ripple_start_t')
    process_and_load_file(session_dict, 'laps_weighted_corr_merged_df', final_sessions_loaded_laps_wcorr_dict, session_str, curr_session_t_delta, 'lap_start_t')
    process_and_load_file(session_dict, 'ripple_weighted_corr_merged_df', final_sessions_loaded_ripple_wcorr_dict, session_str, curr_session_t_delta, 'ripple_start_t')
    
    # process_and_load_file(session_dict, 'laps_all_scores_merged_df', final_sessions_loaded_laps_all_scores_dict, session_str, curr_session_t_delta, 'lap_start_t')
    process_and_load_file(session_dict, 'ripple_all_scores_merged_df', final_sessions_loaded_ripple_all_scores_dict, session_str, curr_session_t_delta, 'ripple_start_t')



## Build across_sessions join dataframes:
all_sessions_laps_df: pd.DataFrame = PandasHelpers.safe_concat(list(final_sessions_loaded_laps_dict.values()), axis='index', ignore_index=True)
all_sessions_ripple_df: pd.DataFrame = PandasHelpers.safe_concat(list(final_sessions_loaded_ripple_dict.values()), axis='index', ignore_index=True)
# Add 'epoch_idx' column for compatibility:
if all_sessions_laps_df is not None:
    all_sessions_laps_df['epoch_idx'] = all_sessions_laps_df['lap_idx']
if all_sessions_ripple_df is not None:
    all_sessions_ripple_df['epoch_idx'] = all_sessions_ripple_df['ripple_idx']

# *_time_bin marginals:
all_sessions_laps_time_bin_df: pd.DataFrame = PandasHelpers.safe_concat(list(final_sessions_loaded_laps_time_bin_dict.values()), axis='index', ignore_index=True)
all_sessions_ripple_time_bin_df: pd.DataFrame = PandasHelpers.safe_concat(list(final_sessions_loaded_ripple_time_bin_dict.values()), axis='index', ignore_index=True)

# NEW ________________________________________________________________________________________________________________ #
all_sessions_simple_pearson_laps_df: pd.DataFrame = PandasHelpers.safe_concat(list(final_sessions_loaded_simple_pearson_laps_dict.values()), axis='index', ignore_index=True)
all_sessions_simple_pearson_ripple_df: pd.DataFrame = PandasHelpers.safe_concat(list(final_sessions_loaded_simple_pearson_ripple_dict.values()), axis='index', ignore_index=True)

if len(final_sessions_loaded_laps_wcorr_dict) > 0:
    all_sessions_wcorr_laps_df: pd.DataFrame = PandasHelpers.safe_concat(list(final_sessions_loaded_laps_wcorr_dict.values()), axis='index', ignore_index=True)
else:
    all_sessions_wcorr_laps_df = None # empty df would be better

all_sessions_wcorr_ripple_df: pd.DataFrame = PandasHelpers.safe_concat(list(final_sessions_loaded_ripple_wcorr_dict.values()), axis='index', ignore_index=True)

# `*_all_scores_*`: __________________________________________________________________________________________________ #
# all_sessions_all_score_laps_df: pd.DataFrame = PandasHelpers.safe_concat(list(final_sessions_loaded_laps_all_scores_dict.values()), axis='index', ignore_index=True)
all_sessions_all_scores_ripple_df: pd.DataFrame = PandasHelpers.safe_concat(list(final_sessions_loaded_ripple_all_scores_dict.values()), axis='index', ignore_index=True)


dfs_list = (all_sessions_laps_df, all_sessions_ripple_df, all_sessions_laps_time_bin_df, all_sessions_ripple_time_bin_df)
for a_df in dfs_list:
    if a_df is not None:
        if 'time_bin_size' not in a_df:
            print('Uh-oh! time_bin_size is missing! This must be old exports!')
            print(f'\tTry to determine the time_bin_size from the filenames: {csv_sessions}')
            ## manual correction UwU
            time_bin_size: float = 0.025
            print(f'WARNING! MANUAL OVERRIDE TIME BIN SIZE SET: time_bin_size = {time_bin_size}. Assigning to dataframes....')
            a_df['time_bin_size'] = time_bin_size
        else:
            # Filter rows based on column: 'time_bin_size'
            a_df = a_df[a_df['time_bin_size'].notna()]


# if 'time_bin_size' not in all_sessions_laps_df:
#     print('Uh-oh! time_bin_size is missing! This must be old exports!')
#     print(f'\tTry to determine the time_bin_size from the filenames: {csv_sessions}')
#     ## manual correction UwU
#     time_bin_size: float = 0.025
#     print(f'WARNING! MANUAL OVERRIDE TIME BIN SIZE SET: time_bin_size = {time_bin_size}. Assigning to dataframes....')
#     all_sessions_laps_df['time_bin_size'] = time_bin_size
#     all_sessions_ripple_df['time_bin_size'] = time_bin_size
#     all_sessions_laps_time_bin_df['time_bin_size'] = time_bin_size
#     all_sessions_ripple_time_bin_df['time_bin_size'] = time_bin_size
#     print(f'\tdone.')
# else:
#     # Filter rows based on column: 'time_bin_size'
#     all_sessions_laps_df = all_sessions_laps_df[all_sessions_laps_df['time_bin_size'].notna()]
#     all_sessions_ripple_df = all_sessions_ripple_df[all_sessions_ripple_df['time_bin_size'].notna()]
#     all_sessions_laps_time_bin_df = all_sessions_laps_time_bin_df[all_sessions_laps_time_bin_df['time_bin_size'].notna()]
#     all_sessions_ripple_time_bin_df = all_sessions_ripple_time_bin_df[all_sessions_ripple_time_bin_df['time_bin_size'].notna()]

all_sessions_laps_df, all_sessions_ripple_df, all_sessions_laps_time_bin_df, all_sessions_ripple_time_bin_df = [_common_cleanup_operations(a_df) for a_df in (all_sessions_laps_df, all_sessions_ripple_df, all_sessions_laps_time_bin_df, all_sessions_ripple_time_bin_df)]
all_sessions_simple_pearson_laps_df, all_sessions_simple_pearson_ripple_df, all_sessions_wcorr_laps_df, all_sessions_wcorr_ripple_df = [_common_cleanup_operations(a_df) for a_df in (all_sessions_simple_pearson_laps_df, all_sessions_simple_pearson_ripple_df, all_sessions_wcorr_laps_df, all_sessions_wcorr_ripple_df)]

# all_sessions_all_score_laps_df, all_sessions_all_scores_ripple_df = [_common_cleanup_operations(a_df) for a_df in (all_sessions_all_score_laps_df, all_sessions_all_scores_ripple_df)]
# all_sessions_all_score_laps_df = _common_cleanup_operations(all_sessions_all_score_laps_df)
all_sessions_all_scores_ripple_df = _common_cleanup_operations(all_sessions_all_scores_ripple_df)

all_sessions_simple_pearson_laps_df: pd.DataFrame = DecoderDecodedEpochsResult.merge_decoded_epochs_result_dfs(all_sessions_simple_pearson_laps_df, all_sessions_wcorr_laps_df, should_drop_directional_columns=False, start_t_idx_name='delta_aligned_start_t')
all_sessions_simple_pearson_ripple_df: pd.DataFrame = DecoderDecodedEpochsResult.merge_decoded_epochs_result_dfs(all_sessions_simple_pearson_ripple_df, all_sessions_wcorr_ripple_df, should_drop_directional_columns=False, start_t_idx_name='ripple_start_t')

# all_sessions_laps_time_bin_df # 601845 rows × 9 column

collected_outputs_directory: /Users/pho/Dropbox (University of Michigan)/MED-DibaLabDropbox/Data/Pho/Outputs/output/collected_outputs
ERR: Could not parse filename: "2024-01-02_GL_831pm-11-02_17-46-44_merged_complete_epoch_stats_df"
ERR: Could not parse filename: "2024-01-03_GL_527am-fet11-01_12-58-54_merged_complete_epoch_stats_df"
ERR: Could not parse filename: "2024-01-03_GL_527am-11-03_12-3-25_merged_complete_epoch_stats_df"
ERR: Could not parse filename: "2024-01-02_GL_831pm-2006-4-09_16-40-54_merged_complete_epoch_stats_df"
ERR: Could not parse filename: "2024-01-03_GL_527am-2006-4-09_17-29-30_merged_complete_epoch_stats_df"
ERR: Could not parse filename: "2024-01-02_GL_831pm-2006-6-09_1-22-43_merged_complete_epoch_stats_df"
ERR: Could not parse filename: "2024-01-02_GL_831pm-2006-6-12_15-55-31_merged_complete_epoch_stats_df"
ERR: Could not parse filename: "2024-01-02_GL_831pm-2006-4-10_12-25-50_merged_complete_epoch_stats_df"
ERR: Could not parse filename: "2024-01-03_GL_527am-2

In [ ]:
csv_sessions

In [ ]:
print(list(all_sessions_all_scores_ripple_df.columns)) # ['start', 'stop', 'label', 'duration', 'is_user_annotated_epoch', 'is_valid_epoch', 'P_LR', 'P_RL', 'P_Long', 'P_Short', 'P_Long_LR', 'score_long_LR', 'velocity_long_LR', 'intercept_long_LR', 'speed_long_LR', 'wcorr_long_LR', 'pearsonr_long_LR', 'travel_long_LR', 'coverage_long_LR', 'jump_long_LR', 'longest_sequence_length_ratio_long_LR', 'direction_change_bin_ratio_long_LR', 'congruent_dir_bins_ratio_long_LR', 'total_congruent_direction_change_long_LR', 'P_Long_RL', 'score_long_RL', 'velocity_long_RL', 'intercept_long_RL', 'speed_long_RL', 'wcorr_long_RL', 'pearsonr_long_RL', 'travel_long_RL', 'coverage_long_RL', 'jump_long_RL', 'longest_sequence_length_ratio_long_RL', 'direction_change_bin_ratio_long_RL', 'congruent_dir_bins_ratio_long_RL', 'total_congruent_direction_change_long_RL', 'P_Short_LR', 'score_short_LR', 'velocity_short_LR', 'intercept_short_LR', 'speed_short_LR', 'wcorr_short_LR', 'pearsonr_short_LR', 'travel_short_LR', 'coverage_short_LR', 'jump_short_LR', 'longest_sequence_length_ratio_short_LR', 'direction_change_bin_ratio_short_LR', 'congruent_dir_bins_ratio_short_LR', 'total_congruent_direction_change_short_LR', 'P_Short_RL', 'score_short_RL', 'velocity_short_RL', 'intercept_short_RL', 'speed_short_RL', 'wcorr_short_RL', 'pearsonr_short_RL', 'travel_short_RL', 'coverage_short_RL', 'jump_short_RL', 'longest_sequence_length_ratio_short_RL', 'direction_change_bin_ratio_short_RL', 'congruent_dir_bins_ratio_short_RL', 'total_congruent_direction_change_short_RL', 'ripple_start_t', 'long_best_travel', 'short_best_travel', 'travel_diff', 'long_best_coverage', 'short_best_coverage', 'coverage_diff', 'long_best_jump', 'short_best_jump', 'jump_diff', 'long_best_longest_sequence_length_ratio', 'short_best_longest_sequence_length_ratio', 'longest_sequence_length_ratio_diff', 'long_best_direction_change_bin_ratio', 'short_best_direction_change_bin_ratio', 'direction_change_bin_ratio_diff', 'long_best_congruent_dir_bins_ratio', 'short_best_congruent_dir_bins_ratio', 'congruent_dir_bins_ratio_diff', 'long_best_total_congruent_direction_change', 'short_best_total_congruent_direction_change', 'total_congruent_direction_change_diff', 'session_name', 'time_bin_size', 'delta_aligned_start_t']
# ['long_best_total_congruent_direction_change', 'short_best_total_congruent_direction_change']
['longest_sequence_length_ratio_diff', 'direction_change_bin_ratio_diff', 'congruent_dir_bins_ratio_diff', 'total_congruent_direction_change_diff']

In [ ]:
all_sessions_simple_pearson_ripple_df # 3138 rows × 24 columns

In [ ]:
all_sessions_simple_pearson_laps_df # 931 rows × 25 columns

In [ ]:
all_sessions_ripple_df # 17592 rows × 10 columns

In [ ]:
csv_files

In [3]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import export_across_session_CSVs

# TODAY_DAY_DATE: str = f"2024-03-18_Apogee"
final_output_path = Path("../output/").resolve()

final_csv_export_paths = export_across_session_CSVs(final_output_path=final_output_path, TODAY_DAY_DATE=TODAY_DAY_DATE,
                                                    all_sessions_laps_df=all_sessions_laps_df,  all_sessions_ripple_df=all_sessions_ripple_df,  all_sessions_laps_time_bin_df=all_sessions_laps_time_bin_df,  all_sessions_ripple_time_bin_df=all_sessions_ripple_time_bin_df, 
                                                    all_sessions_simple_pearson_laps_df=all_sessions_simple_pearson_laps_df,  all_sessions_simple_pearson_ripple_df=all_sessions_simple_pearson_ripple_df,
                                                    all_sessions_all_scores_ripple_df=all_sessions_all_scores_ripple_df,  all_sessions_all_scores_laps_df=None,
                                                    )
final_csv_export_paths

{'AcrossSession_Laps_per-Epoch': PosixPath('/Users/pho/repo/Pho Secondary Workspace/Spike3DEnv/Spike3DWorkEnv/Spike3D/output/2024-03-21_Mac_AcrossSession_Laps_per-Epoch.csv'),
 'AcrossSession_Ripple_per-Epoch': PosixPath('/Users/pho/repo/Pho Secondary Workspace/Spike3DEnv/Spike3DWorkEnv/Spike3D/output/2024-03-21_Mac_AcrossSession_Ripple_per-Epoch.csv'),
 'AcrossSession_Laps_per-TimeBin': PosixPath('/Users/pho/repo/Pho Secondary Workspace/Spike3DEnv/Spike3DWorkEnv/Spike3D/output/2024-03-21_Mac_AcrossSession_Laps_per-TimeBin.csv'),
 'AcrossSession_Ripple_per-TimeBin': PosixPath('/Users/pho/repo/Pho Secondary Workspace/Spike3DEnv/Spike3DWorkEnv/Spike3D/output/2024-03-21_Mac_AcrossSession_Ripple_per-TimeBin.csv'),
 'AcrossSession_SimplePearson_Laps_per-Epoch': PosixPath('/Users/pho/repo/Pho Secondary Workspace/Spike3DEnv/Spike3DWorkEnv/Spike3D/output/2024-03-21_Mac_AcrossSession_SimplePearson_Laps_per-Epoch.csv'),
 'AcrossSession_SimplePearson_Ripple_per-Epoch': PosixPath('/Users/pho/repo/

In [ ]:
all_sessions_all_scores_ripple_df




In [ ]:

all_sessions_simple_pearson_ripple_df

In [ ]:
## find all columns starting with 'wcorr': wcorr_long_LR
sub_string: str = 'wcorr'
sub_string: str = 'pearsonr'
columns_list = list(all_sessions_simple_pearson_ripple_df.columns)
matching_columns = [s for s in columns_list if sub_string in s]
print(matching_columns) # ['wcorr_long_LR', 'wcorr_long_RL', 'wcorr_short_LR', 'wcorr_short_RL', 'long_best_wcorr', 'short_best_wcorr', 'wcorr_abs_diff']

['long_LR_pf_peak_x_pearsonr', 'long_RL_pf_peak_x_pearsonr', 'short_LR_pf_peak_x_pearsonr', 'short_RL_pf_peak_x_pearsonr', 'long_best_pf_peak_x_pearsonr', 'short_best_pf_peak_x_pearsonr', 'pearsonr_abs_diff']
['wcorr_long_LR', 'wcorr_long_RL', 'wcorr_short_LR', 'wcorr_short_RL', 'long_best_wcorr', 'short_best_wcorr', 'wcorr_abs_diff']

assert np.shape(all_sessions_simple_pearson_ripple_df)[0] == np.shape(all_sessions_all_scores_ripple_df)[0], f"np.shape(all_sessions_all_scores_ripple_df)[0]: {np.shape(all_sessions_all_scores_ripple_df)[0]} != np.shape(all_sessions_simple_pearson_ripple_df)[0]: {np.shape(all_sessions_simple_pearson_ripple_df)[0]}"



In [ ]:
['long_LR_pf_peak_x_pearsonr', 'long_RL_pf_peak_x_pearsonr', 'short_LR_pf_peak_x_pearsonr', 'short_RL_pf_peak_x_pearsonr', 'long_best_pf_peak_x_pearsonr', 'short_best_pf_peak_x_pearsonr', 'pearsonr_abs_diff']
['wcorr_long_LR', 'wcorr_long_RL', 'wcorr_short_LR', 'wcorr_short_RL', 'long_best_wcorr', 'short_best_wcorr', 'wcorr_abs_diff']

# 2024-03-02 - Get only the user-annotated ripples:

In [4]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import _split_user_annotated_ripple_df

## Bump
# input_df = all_sessions_simple_pearson_ripple_df
# input_df = all_sessions_all_scores_ripple_df

all_sessions_all_scores_ripple_df, (valid_ripple_df, invalid_ripple_df), (user_approved_ripple_df, user_approved_ripple_df) = _split_user_annotated_ripple_df(all_sessions_all_scores_ripple_df)


## 2024-03-14 - 'is_valid_epoch' column
# 'is_valid_epoch'
## OUTPUTS: valid_ripple_df, invalid_ripple_df, user_approved_ripple_df, user_rejected_ripple_df, (user_annotated_epoch_unique_session_names, unannotated_session_names)
user_approved_ripple_df

user_annotated_epoch_unique_session_names: ['kdiba_pin01_one_fet11-01_12-58-54' 'kdiba_pin01_one_11-02_19-28-0'
 'kdiba_pin01_one_11-02_17-46-44' 'kdiba_gor01_two_2006-6-09_22-24-40'
 'kdiba_gor01_two_2006-6-08_21-16-25' 'kdiba_gor01_two_2006-6-07_16-40-19'
 'kdiba_gor01_one_2006-6-12_15-55-31' 'kdiba_gor01_one_2006-6-09_1-22-43'
 'kdiba_gor01_one_2006-6-08_14-26-15']
unannotated_session_names: set()
n_input_df_rows: 3866
	 n_valid_df_rows: 2399
	 n_invalid_df_rows: 1467
n_input_df_rows: 2399
	 n_user_approved_df_rows: 415
	 n_user_rejected_df_rows: 1984


,start,stop,label,duration,P_LR,P_RL,P_Long,P_Short,P_Long_LR,score_long_LR,velocity_long_LR,intercept_long_LR,speed_long_LR,wcorr_long_LR,pearsonr_long_LR,travel_long_LR,coverage_long_LR,jump_long_LR,longest_sequence_length_ratio_long_LR,direction_change_bin_ratio_long_LR,congruent_dir_bins_ratio_long_LR,total_congruent_direction_change_long_LR,P_Long_RL,score_long_RL,velocity_long_RL,intercept_long_RL,speed_long_RL,wcorr_long_RL,pearsonr_long_RL,travel_long_RL,coverage_long_RL,jump_long_RL,longest_sequence_length_ratio_long_RL,direction_change_bin_ratio_long_RL,congruent_dir_bins_ratio_long_RL,total_congruent_direction_change_long_RL,P_Short_LR,score_short_LR,velocity_short_LR,intercept_short_LR,speed_short_LR,wcorr_short_LR,pearsonr_short_LR,travel_short_LR,coverage_short_LR,jump_short_LR,longest_sequence_length_ratio_short_LR,direction_change_bin_ratio_short_LR,congruent_dir_bins_ratio_short_LR,total_congruent_direction_change_short_LR,...,direction_change_bin_ratio_short_RL,congruent_dir_bins_ratio_short_RL,total_congruent_direction_change_short_RL,ripple_start_t,long_best_P_decoder,short_best_P_decoder,P_decoder_diff,long_best_score,short_best_score,score_diff,long_best_velocity,short_best_velocity,velocity_diff,long_best_intercept,short_best_intercept,intercept_diff,long_best_speed,short_best_speed,speed_diff,long_best_wcorr,short_best_wcorr,wcorr_diff,long_best_pearsonr,short_best_pearsonr,pearsonr_diff,long_best_travel,short_best_travel,travel_diff,long_best_coverage,short_best_coverage,coverage_diff,long_best_jump,short_best_jump,jump_diff,long_best_longest_sequence_length_ratio,short_best_longest_sequence_length_ratio,longest_sequence_length_ratio_diff,long_best_direction_change_bin_ratio,short_best_direction_change_bin_ratio,direction_change_bin_ratio_diff,long_best_congruent_dir_bins_ratio,short_best_congruent_dir_bins_ratio,congruent_dir_bins_ratio_diff,long_best_total_congruent_direction_change,short_best_total_congruent_direction_change,total_congruent_direction_change_diff,session_name,time_bin_size,delta_aligned_start_t,pre_post_delta_category
0,72.773154,72.958611,28,0.185457,0.179749,0.820251,0.835448,0.164552,0.150171,0.542140,184.961887,22.719851,184.961887,0.619718,NaN,0.272857,0.238095,0.003392,0.428571,0.666667,0.666667,197.537004,0.685276,0.479469,87.917239,56.071138,87.917239,0.269071,NaN,0.417994,0.365079,0.003995,0.571429,0.500000,0.833333,350.348649,0.029578,0.534620,415.482224,44.889421,415.482224,0.802748,NaN,0.159610,0.317460,0.001904,0.714286,0.333333,0.833333,134.176078,...,0.500000,0.666667,134.176078,72.773154,0.150171,0.029578,0.120593,0.542140,0.534620,0.007521,184.961887,415.482224,-230.520337,22.719851,44.889421,-22.169570,184.961887,415.482224,-230.520337,0.619718,0.802748,-0.183030,NaN,NaN,NaN,0.272857,0.159610,0.113247,0.238095,0.317460,-0.079365,0.003392,0.001904,0.001487,0.428571,0.714286,-0.285714,0.666667,0.333333,0.333333,0.666667,0.833333,-0.166667,197.537004,134.176078,6.336093e+01,kdiba_pin01_one_fet11-01_12-58-54,0.025,-1984.452794,pre-delta
1,127.406680,127.814365,50,0.407685,0.424923,0.575077,0.678501,0.321499,0.288311,0.252368,239.782583,123.831203,239.782583,0.240120,0.600822,0.306529,0.428571,0.003542,0.187500,0.666667,0.466667,547.885654,0.390190,0.254432,93.514037,184.419316,93.514037,0.291272,0.490229,0.306529,0.428571,0.003618,0.312500,0.533333,0.533333,555.339880,0.136613,0.320034,347.554815,12.505946,347.554815,0.078358,0.556009,0.427928,0.333333,0.003696,0.187500,0.666667,0.466667,469.616274,...,0.666667,0.400000,0.000000,127.406680,0.288311,0.136613,0.151698,0.252368,0.320034,-0.067666,239.782583,347.554815,-107.772232,123.831203,12.505946,111.325257,239.782583,347.554815,-107.772232,0.240120,0.078358,0.161763,0.600822,0.556009,0.044812,0.306529,0.427928,-0.121399,0.428571,0.333333,0.095238,0.003542,0.003696,-0.000154,0.187500,0.187500,0.000000,0.666667,0.666667,0.000000,0.466667,0.466667,0.000000,547.885654,469.616274,7.826938e+01,kdiba_pin01_one_fet11-01_12-58

In [ ]:
user_approved_ripple_df

In [ ]:
[col.split('_long_LR') for col in list(user_approved_ripple_df.columns)]



In [ ]:
## order columns for reasonability
['long_best_coverage', 'short_best_coverage', 'coverage_diff']

['coverage_short_RL']

['_long_LR', '_long_RL', '_short_LR', '_short_RL']
['long_best_', 'short_best_']
['_diff']


# PandasHelpers.reordering_columns(


In [ ]:
## want to compare difference between user approved and user rejected ripples

user_approved_ripple_df
user_rejected_ripple_df
# OUTPUTS: all_sessions_user_annotated_ripple_df, user_approved_ripple_df, user_rejected_ripple_df

In [ ]:
## Want to test for differences between long/short among only user approved ripples:

0.025


In [ ]:
df: pd.DataFrame = deepcopy(all_sessions_user_annotated_ripple_df)

### 2024-02-29 - 4pm - Filter the events for those meeting wcorr criteria:


In [ ]:
# df: pd.DataFrame = deepcopy(all_sessions_user_annotated_ripple_df)
df: pd.DataFrame = deepcopy(valid_ripple_df) # valid epochs, but not just those that the user approved
# df: pd.DataFrame = deepcopy(user_approved_ripple_df)

## INPUTS: df

min_wcorr_threshold: float = 0.33
min_wcorr_diff_threshold: float = 0.2

# is_included_large_wcorr_diff = np.any((df[['wcorr_abs_diff']].abs() > min_wcorr_diff_threshold), axis=1)
is_included_large_wcorr_diff = np.any((df[['wcorr_diff']].abs() > min_wcorr_diff_threshold), axis=1)
is_included_high_wcorr = np.any((df[['long_best_wcorr', 'short_best_wcorr']].abs() > min_wcorr_threshold), axis=1)

df = df[is_included_high_wcorr]
df

# wcorr_long_LR


# Plotting via Plotly

In [5]:
import plotly.io as pio
template: str = 'plotly_dark' # set plotl template
# template: str = 'plotly_white'
pio.templates.default = template
from PIL import Image

from pyphocorehelpers.programming_helpers import copy_image_to_clipboard
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import _helper_build_figure, plotly_pre_post_delta_scatter, plot_across_sessions_scatter_results


# 1650, 446
fig_size_kwargs = {'width': 1650, 'height': 480}

In [6]:
concatenated_ripple_df = deepcopy(all_sessions_all_scores_ripple_df).infer_objects()

## Add plotly helper columns:
concatenated_ripple_df['pre_post_delta_category'] = 'post-delta'
concatenated_ripple_df['pre_post_delta_category'][concatenated_ripple_df['delta_aligned_start_t'] < 0.0] = 'pre-delta'


{'pre_post_delta_category': ['pre-delta', 'post-delta']

}

ripple_title_prefix = "User Annotated Sessions Ripples"

# concatenated_ripple_df = deepcopy(user_approved_ripple_df)
# ripple_title_prefix = "User Selected Ripples Only"

# concatenated_ripple_df = deepcopy(user_rejected_ripple_df)
# ripple_title_prefix = "User Rejected Ripples Only"

# concatenated_ripple_df = deepcopy(df)


# Create a bubble chart for ripples
enabled_time_bin_sizes = None
ripple_num_unique_sessions: int = concatenated_ripple_df.session_name.nunique(dropna=True) # number of unique sessions, ignoring the NA entries
ripple_num_unique_time_bins: int = concatenated_ripple_df.time_bin_size.nunique(dropna=True)
ripple_title_string_suffix: str = f'{ripple_num_unique_sessions} Sessions'
ripple_title: str = f"{ripple_title_prefix} - {ripple_title_string_suffix}"
fig_ripples = _helper_build_figure(data_results_df=concatenated_ripple_df, histogram_bins=25, earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end, enabled_time_bin_sizes=enabled_time_bin_sizes, main_plot_mode='default', title=ripple_title)
fig_ripples.update_layout(fig_size_kwargs)
fig_ripples.show()
# fig_to_clipboard(fig_ripples)

/var/folders/c7/74vbnwdj7531qy2glkcjq6jr0000gn/T/ipykernel_7173/87855474.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concatenated_ripple_df['pre_post_delta_category'][concatenated_ripple_df['delta_aligned_start_t'] < 0.0] = 'pre-delta'


num_unique_sessions: 9, num_unique_time_bins: 1
main_plot_mode: default
num_cols: 2


In [7]:
histogram_bins = 25
# concatenated_ripple_df = deepcopy(df)

# ripple_title_prefix = 'test'
# # Create a bubble chart for ripples
# ripple_num_unique_sessions: int = concatenated_ripple_df.session_name.nunique(dropna=True) # number of unique sessions, ignoring the NA entries
# # ripple_num_unique_time_bins: int = concatenated_ripple_df.time_bin_size.nunique(dropna=True)
# ripple_title_string_suffix: str = f'{ripple_num_unique_sessions} Sessions'
# ripple_title: str = f"{ripple_title_prefix} - {ripple_title_string_suffix}"
# fig_ripples = _helper_build_figure(data_results_df=concatenated_ripple_df, histogram_bins=histogram_bins, earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end, enabled_time_bin_sizes=None,
#                                     main_plot_mode='separate_row_per_session', title=ripple_title)

# concatenated_ripple_df = deepcopy(concatenated_ripple_df)
concatenated_ripple_df = deepcopy(all_sessions_all_scores_ripple_df)

# ['longest_sequence_length_ratio_diff', 'direction_change_bin_ratio_diff', 'congruent_dir_bins_ratio_diff', 'total_congruent_direction_change_diff']


# variable_name = 'P_Long'
variable_name = 'P_Short' # Shows expected effect - short-only replay prior to delta and then split replays post-delta
# variable_name = 'long_best_pf_peak_x_pearsonr'
# variable_name = 'long_best_jump'
# variable_name = 'wcorr_abs_diff'
# variable_name = 'pearsonr_abs_diff'
# variable_name = 'direction_change_bin_ratio_diff'
# variable_name = 'longest_sequence_length_ratio_diff'
# variable_name = 'long_best_longest_sequence_length_ratio'
# variable_name = 'long_best_congruent_dir_bins_ratio'
# variable_name = 'congruent_dir_bins_ratio_diff'
# variable_name = 'total_congruent_direction_change_diff'
# variable_name = 'long_best_congruent_dir_bins_ratio'
# variable_name = 'long_best_direction_change_bin_ratio'
# variable_name = 'long_best_congruent_dir_bins_ratio'
# 'color':'is_user_annotated_epoch'
# 'color': 'is_user_annotated_epoch', 
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'color':"is_user_annotated_epoch", 'title': f"'{variable_name}'"} # , 'color': 'time_bin_size', 'range_y': [-1.0, 1.0], 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size', 'is_user_annotated_epoch':'user_sel'}
# hist_kwargs = dict(color="time_bin_size")
hist_kwargs = dict(color="is_user_annotated_epoch") # , histnorm='probability density'
new_fig_ripples = plotly_pre_post_delta_scatter(data_results_df=concatenated_ripple_df, out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, hist_kwargs=hist_kwargs, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end))
fig_ripples.update_layout(fig_size_kwargs)

_extras_output_dict = {}
_extras_output_dict["y_mid_line"] = new_fig_ripples.add_hline(y=0.5, line=dict(color="rgba(0.8,0.8,0.8,.75)", width=2), row='all', col='all')

# fig_to_clipboard(new_fig_ripples, **fig_size_kwargs)
new_fig_ripples


num_unique_sessions: 9, num_unique_time_bins: 1
len(pre_delta_fig.data): 2


In [ ]:
# new_fig_ripples.get_subplot(
# import kaleido

# new_fig_ripples.layout
new_fig_ripples.layout.width

# from kaleido._version import __version__ # '0.1.0.post1'
# __version__

color_options = ["is_user_annotated_epoch", "pre_post_delta_category"]

variable_options = {'pre_post_delta_category': ['pre-delta', 'post-delta']
}

concatenated_ripple_df

In [14]:
variable_name = 'short_best_direction_change_bin_ratio'
# variable_name = 'long_best_congruent_dir_bins_ratio'
# variable_name = 'short_best_wcorr'
# 'color':'is_user_annotated_epoch'
# 'color': 'is_user_annotated_epoch', 
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'color':"pre_post_delta_category", 'title': f"'{variable_name}'"} # , 'color': 'time_bin_size', 'range_y': [-1.0, 1.0], , 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size', 'is_user_annotated_epoch':'user_sel'}
# hist_kwargs = dict(color="time_bin_size")
hist_kwargs = dict(color="pre_post_delta_category") # , histnorm='probability density'
new_fig_ripples = plotly_pre_post_delta_scatter(data_results_df=concatenated_ripple_df, out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, hist_kwargs=hist_kwargs, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end))
_extras_output_dict = {}
_extras_output_dict["y_mid_line"] = new_fig_ripples.add_hline(y=0.0, line=dict(color="rgba(0.8,0.8,0.8,.75)", width=2), row='all', col='all')
new_fig_ripples.update_layout(fig_size_kwargs)
new_fig_ripples

num_unique_sessions: 9, num_unique_time_bins: 1
len(pre_delta_fig.data): 1


In [ ]:
# variable_name = 'total_congruent_direction_change_diff'
# variable_name = 'long_best_congruent_dir_bins_ratio'
# variable_name = 'long_best_total_congruent_direction_change'
variable_name = 'wcorr_diff'
# variable_name = 'long_best_wcorr'
# 'color':'is_user_annotated_epoch'
# 'color': 'is_user_annotated_epoch', 
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'color':"is_user_annotated_epoch", 'title': f"'{variable_name}'", 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size', 'is_user_annotated_epoch':'user_sel'}} # , 'color': 'time_bin_size', 'range_y': [-1.0, 1.0], 
# hist_kwargs = dict(color="time_bin_size")
hist_kwargs = dict(color="is_user_annotated_epoch") # , histnorm='probability density'
new_fig_ripples = plotly_pre_post_delta_scatter(data_results_df=concatenated_ripple_df, out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, hist_kwargs=hist_kwargs, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end))
_extras_output_dict = {}
_extras_output_dict["y_mid_line"] = new_fig_ripples.add_hline(y=0.0, line=dict(color="rgba(0.8,0.8,0.8,.75)", width=2), row='all', col='all')
new_fig_ripples.update_layout(fig_size_kwargs)
new_fig_ripples


## Stats Tests

In [9]:
from pyphocorehelpers.indexing_helpers import partition, partition_df, partition_df_dict
import scipy.stats as stats

# ['pre-delta', 'post-delta']
analysis_df = deepcopy(valid_ripple_df[["delta_aligned_start_t", "pre_post_delta_category", "short_best_wcorr", 'P_Short']]).dropna(subset=["pre_post_delta_category", "P_Short"])
# partition_df(analysis_df, partitionColumn='pre_post_delta_category')

# _partition_values, (_pre_delta_df, _post_delta_df) = partition(analysis_df, partitionColumn='pre_post_delta_category') # use `valid_ripple_df` instead of the original dataframe to only get those which are valid.
_pre_post_delta_partition_df_dict = partition_df_dict(analysis_df, partitionColumn='pre_post_delta_category')
_pre_delta_df = _pre_post_delta_partition_df_dict['pre-delta']
_post_delta_df = _pre_post_delta_partition_df_dict['post-delta']


stats_variable_name: str = 'P_Short'
actual_diff_means = np.nanmean(_post_delta_df[stats_variable_name].to_numpy()) - np.nanmean(_pre_delta_df[stats_variable_name].to_numpy())
print(f'stats_variable_name: {stats_variable_name}: actual_diff_means: {actual_diff_means}')

# pre_delta_ripple_df: pd.DataFrame = _partitioned_analysis_dfs[True].drop(columns=['pre_post_delta_category']).reset_index(drop=True)
# post_delta_ripple_df: pd.DataFrame = _partitioned_analysis_dfs[False].drop(columns=['pre_post_delta_category']).reset_index(drop=True)
_pre_delta_df
_post_delta_df

stats_variable_name: P_Short: actual_diff_means: 0.13763328444859513


,delta_aligned_start_t,pre_post_delta_category,short_best_wcorr,P_Short
396,0.449255,post-delta,0.019656,0.553403
397,24.821231,post-delta,0.044019,0.722681
398,28.194286,post-delta,-0.569453,0.481033
399,30.416262,post-delta,-0.072889,0.355320
400,31.264656,post-delta,0.451736,0.489578
...,...,...,...,...
2394,823.097073,post-delta,-0.276354,0.747061
2395,832.334111,post-delta,-0.570605,0.393241
2396,833.537352,post-delta,-0.320426,0.564155
2397,839.584212,post-delta,0.332299,0.838134


In [10]:
n_shuffles: int = 10000
stats_variable_name: str = 'P_Short'

shuffle_results = []
## INPUT: n_shuffles, analysis_df, stats_variable_name
shuffled_analysis_df = deepcopy(analysis_df)
for i in np.arange(n_shuffles):
    # shuffled_analysis_df[stats_variable_name] = shuffled_analysis_df[stats_variable_name].sample(frac=1).to_numpy() # .reset_index(drop=True)
    shuffled_analysis_df['pre_post_delta_category'] = shuffled_analysis_df.sample(frac=1)['pre_post_delta_category'].to_numpy()
    _shuffled_pre_post_delta_partition_df_dict = partition_df_dict(shuffled_analysis_df, partitionColumn='pre_post_delta_category')
    _shuffled_pre_delta_df = _shuffled_pre_post_delta_partition_df_dict['pre-delta']
    _shuffled_post_delta_df = _shuffled_pre_post_delta_partition_df_dict['post-delta']

    _diff_mean = np.nanmean(_shuffled_post_delta_df[stats_variable_name].to_numpy()) - np.nanmean(_shuffled_pre_delta_df[stats_variable_name].to_numpy())
    shuffle_results.append(_diff_mean)

shuffle_results = np.array(shuffle_results)
shuffle_results

array([ 0.00415046, -0.00651542, -0.00065668, ...,  0.01157959,
        0.00080947,  0.00593886])

In [ ]:
np.sum(actual_diff_means > np.abs(shuffle_results))

In [11]:
# Create the data for two groups
# group1 = np.random.rand(25)
# group2 = np.random.rand(20)

stats_variable_name: str = 'P_Short'
group1 = _pre_delta_df[stats_variable_name].to_numpy()
group2 = _post_delta_df[stats_variable_name].to_numpy()

# perform mann whitney test 
stat, p_value = stats.mannwhitneyu(group1, group2) 
print('Statistics=%.2f, p=%.2f' % (stat, p_value)) 


# Level of significance 
alpha = 0.05
# conclusion 
if p_value < alpha: 
    print('Reject Null Hypothesis (Significant difference between two samples)') 
else: 
    print('Do not Reject Null Hypothesis (No significant difference between two samples)')

# Calculate the sample variances
variance1 = np.var(group1, ddof=1)
variance2 = np.var(group2, ddof=1)
 
print('Variance 1:',variance1)
print('Variance 2:',variance2)

# Calculate the F-statistic
f_value = variance1 / variance2
 
# Calculate the degrees of freedom
df1 = len(group1) - 1
df2 = len(group2) - 1
 
# Calculate the p-value
p_value = stats.f.cdf(f_value, df1, df2)
 
# Print the results
print('Degree of freedom 1:',df1)
print('Degree of freedom 2:',df2)
print("F-statistic:", f_value)
print("p-value:", p_value)


# Statistics=351933.00, p=0.00
# Reject Null Hypothesis (Significant difference between two samples)
# Variance 1: 0.017720245875104713
# Variance 2: 0.02501347759017487
# Degree of freedom 1: 1104
# Degree of freedom 2: 1282
# F-statistic: 0.7084279189577826
# p-value: 1.882791591520268e-09


Statistics=351933.00, p=0.00
Reject Null Hypothesis (Significant difference between two samples)
Variance 1: 0.017720245875104713
Variance 2: 0.02501347759017487
Degree of freedom 1: 1104
Degree of freedom 2: 1282
F-statistic: 0.7084279189577826
p-value: 1.882791591520268e-09


In [ ]:
## User non-selected:
scatter_title = f'user_approved_ripple_df Sevveral Sessions {variable_name}'
# variable_name = 'wcorr_abs_diff'
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'title': scatter_title, 'range_y': [0.0, 1.0], 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size'}} # , 'color': 'time_bin_size'
new_fig_ripples = plotly_pre_post_delta_scatter(data_results_df=deepcopy(user_approved_ripple_df), out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end))
_extras_output_dict["y_mid_line"] = new_fig_ripples.add_hline(y=0.5, line=dict(color="rgba(0.8,0.8,0.8,.75)", width=2), row='all', col='all')
new_fig_ripples.update_layout(fig_size_kwargs)
new_fig_ripples

In [ ]:
# IDEA: The ones with clear replays (diagonal sequences in the decoded posteriors) are by definiition ambiguous, because there's not much difference between the long/short decoders.


In [ ]:
## User non-selected:
scatter_title = f'Non-selected Sevveral Sessions {variable_name}'
# variable_name = 'wcorr_abs_diff'
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'title': scatter_title, 'range_y': [0.0, 1.0], 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size'}} # , 'color': 'time_bin_size'
new_fig_ripples = plotly_pre_post_delta_scatter(data_results_df=deepcopy(user_rejected_ripple_df), out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end))
new_fig_ripples.update_layout(fig_size_kwargs)
new_fig_ripples


In [ ]:
# Laps test
concatenated_ripple_df = deepcopy(all_sessions_simple_pearson_laps_df)

scatter_title = 'Sevveral Sessions'
variable_name = 'wcorr_abs_diff'
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'title': scatter_title, 'range_y': [0.0, 1.0], 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size'}} 
new_fig_ripples = plotly_pre_post_delta_scatter(data_results_df=deepcopy(concatenated_ripple_df), out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end))
new_fig_ripples.update_layout(fig_size_kwargs)
new_fig_ripples


# Plotting functions

In [8]:
from pyphoplacecellanalysis.General.Model.Configs.LongShortDisplayConfig import PlottingHelpers
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import plot_across_sessions_scatter_results

# Example usage:
all_session_figures = plot_across_sessions_scatter_results(collected_outputs_directory, concatenated_laps_df=all_sessions_laps_df, concatenated_ripple_df=all_sessions_ripple_df,
														#    enabled_time_bin_sizes=[0.03, 0.10],
														   earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end,
														   laps_title_prefix=f"Laps", ripple_title_prefix=f"Ripples", save_figures=True, figure_save_extension=['.html','.png'])
fig_laps, fig_ripples = all_session_figures[0]
# fig_laps.update_layout(fig_size_kwargs)
# fig_ripples.update_layout(fig_size_kwargs)

# fig_laps.show()
fig_ripples.show()
# fig_laps.write_html(f"../output/{TODAY_DAY_DATE}_AcrossSession_fig_laps.html")
# fig_ripples.write_html(f"../output/{TODAY_DAY_DATE}_AcrossSession_fig_ripples.html")

ImportError: cannot import name 'plot_across_sessions_scatter_results' from 'pyphoplacecellanalysis.SpecificResults.PendingNotebookCode' (/Users/pho/repo/Pho Secondary Workspace/Spike3DEnv/Spike3DWorkEnv/pyPhoPlaceCellAnalysis/src/pyphoplacecellanalysis/SpecificResults/PendingNotebookCode.py)

In [ ]:
fig_to_clipboard(fig_ripples)

In [ ]:
## time_bin version:
all_time_bin_session_figures = plot_across_sessions_scatter_results(collected_outputs_directory, concatenated_laps_df=all_sessions_laps_time_bin_df, concatenated_ripple_df=all_sessions_ripple_time_bin_df,
														#    enabled_time_bin_sizes=[0.03, 0.10],
														   earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end,
														   main_plot_mode='separate_row_per_session',
														   laps_title_prefix=f"Laps_per_time_bin", ripple_title_prefix=f"Ripples_per_time_bin", save_figures=True, figure_save_extension=['.html','.png'])
fig_time_bin_laps, fig_time_bin_ripples = all_time_bin_session_figures[0]
# fig_time_bin_laps.show()
fig_time_bin_ripples.show()

In [ ]:
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import plot_across_sessions_scatter_results, _helper_build_figure

## Test collapsed histograms-only results:
histograms_only_all_time_bin_session_figures = plot_across_sessions_scatter_results(collected_outputs_directory, concatenated_laps_df=all_sessions_laps_time_bin_df, concatenated_ripple_df=all_sessions_ripple_time_bin_df,
														#    enabled_time_bin_sizes=[0.03, 0.10],
															# enabled_time_bin_sizes=[0.03, 0.058, 0.10], # [0.03 , 0.044, 0.058, 0.072, 0.086, 0.1]
														   earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end,
														   main_plot_mode='default',
														   laps_title_prefix=f"Laps_per_time_bin", ripple_title_prefix=f"Ripples_per_time_bin", save_figures=False, figure_save_extension=['.html','.png'])
histograms_only_fig_time_bin_laps, histograms_only_fig_time_bin_ripples = histograms_only_all_time_bin_session_figures[0]
# histograms_only_fig_time_bin_laps.show()
histograms_only_fig_time_bin_ripples.show()

In [ ]:
import plotly.subplots as sp
import plotly.graph_objs as go

from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import plotly_pre_post_delta_scatter
# data_results_df: pd.DataFrame = deepcopy(all_sessions_laps_df) # all_sessions_laps_time_bin_df
# histogram_bins = 'auto'
histogram_bins: int = 25

new_laps_fig = plotly_pre_post_delta_scatter(data_results_df=deepcopy(all_sessions_laps_df), out_scatter_fig=fig_laps, histogram_bins=histogram_bins)
new_laps_fig

# Matplotlib-based versions:

In [ ]:
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import plot_histograms

matplotlib.use('Qt5Agg')

In [ ]:
# from Spike3D.PendingNotebookCode import plot_stacked_histograms

# You can use it like this:'long_best_dir_simple_pearsonr'
_out0: "MatplotlibRenderPlots" = plot_histograms(all_sessions_laps_time_bin_df, 'Laps', 'All Sessions', "75 ms")
_out1: "MatplotlibRenderPlots" = plot_histograms(all_sessions_ripple_time_bin_df, 'Ripples', 'All Sessions', "75 ms")

In [ ]:


fig_to_clipboard(_out0.figures[0])

In [ ]:
## all_sessions_new_laps_df, all_sessions_new_ripple_df
plot_histograms(all_sessions_new_laps_df, 'New Laps', 'All Sessions', "25 ms")
plot_histograms(all_sessions_new_ripple_df, 'New Ripples', 'All Sessions', "25 ms")


In [ ]:
all_sessions_new_laps_df

In [ ]:
len(all_sessions_new_laps_df['session_name'].unique()) # 10 sessions



In [ ]:
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import plot_stacked_histograms

# You can use it like this:
num_unique_sessions: int = all_sessions_laps_time_bin_df.session_name.nunique(dropna=True) # number of unique sessions, ignoring the NA entries
num_unique_time_bins: int = all_sessions_laps_time_bin_df.time_bin_size.nunique(dropna=True)
plot_stacked_histograms(all_sessions_laps_time_bin_df, 'Laps', f'{num_unique_sessions} Sessions', f"{num_unique_time_bins} tbin sizes")

num_unique_sessions: int = all_sessions_ripple_time_bin_df.session_name.nunique(dropna=True) # number of unique sessions, ignoring the NA entries
num_unique_time_bins: int = all_sessions_ripple_time_bin_df.time_bin_size.nunique(dropna=True)
plot_stacked_histograms(all_sessions_ripple_time_bin_df, 'Ripples', f'{num_unique_sessions} Sessions', f"{num_unique_time_bins} tbin sizes")